In [1]:
import numpy as np

## Definitions:

### Primitive root of unity
Any complex number that yields 1 when raised to some positive integer power n.
1. $ \zeta^n = 1 $
2. $ \zeta^k \neq 1 $ for all integers $ k $ such that $ 0 < k < n $.

$\zeta^n = e^{i(2\pi \frac{k}{n})}$

For 2N-th roots of unity, this means any number $\zeta$ that satisfies $\zeta^{2N}=1$

### Cyclotomic Polynomial
A **cyclotomic polynomial** is a special type of polynomial defined as the unique irreducible polynomial with integer coefficients whose roots are the **primitive roots of unity** $e^{i(2\pi\frac{k}{n})}$, where k runs over the positive integers less than n and coprime to n. The formula is equal to:

$$
\Phi_{n}(x) = \prod_{\substack{1 \leq k \leq n \\ \gcd(k, n) = 1}} \left(x - e^{2i\pi \frac{k}{n}}\right)
$$

**Special Case**: $\Phi_M(X) = X^N + 1$

The specific form $\Phi_M(X) = X^N + 1$ arises when:

- $M = 2N$, i.e., $M$ is an even number and M is a power of 2.
- The roots of $\Phi_{2N}(X)$ are the primitive $2N$-th roots of unity.


#### 2.1. Roots of $X^N + 1$

The equation $X^N + 1 = 0$ has $N$ distinct roots in the complex plane. These roots are the **odd powers** of $\xi$, a primitive $2N$-th root of unity:

$$
\xi = e^{2\pi i / (2N)}, \quad \text{so } \xi^{2k+1} \text{ for } k = 0, 1, \dots, N-1.
$$



#### 2.2. Factorization

The polynomial $X^N + 1$ factors as:

$$
X^N + 1 = \prod_{k=0}^{N-1} \left( X - \xi^{2k+1} \right),
$$

where $\xi^{2k+1}$ are the odd powers of the primitive $2N$-th root of unity $\xi$.



#### 2.3. Minimal Polynomial

When $M = 2N$, the primitive $2N$-th roots of unity are precisely the roots of $X^N + 1$. Hence, the minimal polynomial of the primitive $2N$-th roots of unity is:

$$
\Phi_{2N}(X) = X^N + 1.
$$


### Isomorphism

An isomorphism between two algebraic structures is a bijective homomorphism.

**Addition Preservation**:
$\sigma(m1 + m2) = \sigma(m1) + \sigma(m2)$

**Multiplication Preservation**:
$\sigma(m1 * m2) = \sigma(m1) \circ \sigma(m2)$

- Injectivity: No two distinct polynomials map to the same vector under σ.
- Surjectivity: Every vector in ${C^N}$ (satisfying the necessary properties) is the image of some polynomial under σσ.




In [296]:
from math import sin,cos,pi
import numpy as np

class CKKSEncoder:
    """Basic CKKS encoder to encode complex vectors into polynomials."""
    
    def __init__(self, M: int):
        """Initialization of the encoder for M a power of 2. 
        
        xi, which is an M-th root of unity will, be used as a basis for our computations.
        """
        # self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M
        self.N = M // 2


    def pi_inverse(self, z: np.array) -> np.array:
        """Expands a vector of C^{N/2} by expanding it with its
        complex conjugate."""
        
        z_conjugate = z[::-1]
        z_conjugate = [np.conjugate(x) for x in z_conjugate]
        return np.concatenate([z, z_conjugate])



    def ifft(self, z):
    
        xi_inv = np.exp(2 * np.pi * 1j / self.N)

        a = self.fft(z, xi_inv)
    
        for j in range(self.N):
            a[j] /= self.N
    
        return a

    # Recursive function of FFT
    def fft(self, a, xi):

        n = len(a)
          
        # if input contains just one element
        if n==1:
            return [a[0]]
        
        # Separe coefficients
        Aeven = a[0::2]
        Aodd  = a[1::2]
     
        # Recursive call for even indexed coefficients
        Yeven = self.fft(Aeven, xi**2) 
     
        # Recursive call for odd indexed coefficients
        Yodd = self.fft(Aodd, xi**2)
     
        # for storing values of y0, y1, y2, ..., yn-1.
        Y = [0]*n
    
        for k in range(n//2):
            twiddle = xi**k
             
            Y[k] =  Yeven[k] + twiddle *  Yodd[k]
            Y[k + n//2] =  Yeven[k]  -  twiddle * Yodd[k]
         
        return Y

In [313]:
# EXAMPLE

import numpy as np
from numpy.polynomial import Polynomial
from sympy import ntt

M = 8
N = M // 2



def sigma(p: Polynomial, xi) -> np.array:
    """Decodes a polynomial by applying it to the M-th roots of unity."""

    outputs = []
    N = 4

    # We simply apply the polynomial on the roots
    for i in range(N):
        root = xi ** (2*i)
        print(i+1)
        output = p(root)
        outputs.append(output)
    return np.array(outputs)

# # pol = np.array([2.5+4.440892098500626e-16j,-4.996003610813204e-16+0.7071067811865479j,-3.4694469519536176e-16+0.5000000000000003j, -8.326672684688674e-16+0.7071067811865472j])

# prime = 5

encoder = CKKSEncoder(M)

# print(pol)
xi = np.exp(-2 * np.pi * 1j / M)
pol = np.array([2.5, 1.421875, 2.5, 0.703125])
print(encoder.fft(pol, xi))

# transform = np.fft.fft(pol)
# print ("FFT : ", transform)

# pol = Polynomial([2.5, 1.421875, 2.5, 0.703125])
# xi = np.exp(-2 * np.pi * 1j / M)
# res = sigma(pol, xi)
# print("new: ", res)


# p = encoder.sigma_inverse(b)
# p
# b_reconstructed = encoder.sigma(p)
# b


# encoder.ifft(pol)




[np.complex128(7.125+0j), np.complex128(0.508232998977831-0.508232998977831j), np.complex128(2.875+0j), np.complex128(-0.508232998977831+0.508232998977831j)]


In [286]:
# M = 8
# N = M // 2


# # b = np.array([1, 2, 3, 4])

# delta = 64

# # input vector
# inp = np.array([3+4j, 2-1j])


# # append the conjugate since N=4
# inp = encoder.pi_inverse(inp)

# # scaling 
# inp = inp * delta



# # p = encoder.ifft(inp)
# # print(p)
# # np.dot(delta, p)



# # p
# # xi = np.exp(2 * np.pi * 1j / N)
# # d = encoder.fft(p, xi)
# # d